# <font color='hotpink'> Flask API </font>


* REST: Representational State Transfer
* REST APIs are stateless because, rather than relying on the server remembering previous requests, REST applications require each request to contain all of the information necessary for the server to understand it. Storing session state on the server violates the REST architecture's stateless requirement.
* REST is independent of platform and languages.
* REST is not constrained to one format. Can return XML, JSON, YAML etc.
* SOAP: Simple Object Access Protocol 
* SOAP is by default stateless, but it is possible to make this API stateful.
* REST constraints:
    1. Client-Server: Client need not to know anything about business/data acess layer and server need not to know anything about web UI.
    2. Stateless: Server doesn't need to store any session data.
    3. Cache: Response should be cacheable, if possible. Reduce latency.
    4. Uniform Interface: Should work same as mobile, laptop etc.
    5. Layered System: Should have hierarchical layer, and each layer should not know anything beyond immediate layer. Example as MVP architecture.
* Installation:
```
!pip install flask-restful
```

### API building blocks

* Following Steps are involved in developing a simple API :
    1. Definition of a class derived from Resource Class. This is the major part of building a REST API.
        + A resource in REST is similar to object in Object Oriented Programming or is like an Entity in a Database. 
        + Resource class represents an abstract RESTful resource.
        + Concrete resources should extend from this class and expose methods for each supported HTTP method.
        + If a resource is invoked with an unsupported HTTP method, the API will return a response with status 405 Method Not Allowed.
    2. Definition one or more methods, corresponding to HTTP methods, inside derived Class. The allowed methods are get, put, post, and delete.
    3. Map the resource class with one or more URL's using add_resource function.
* HTTP request methods:
    + Create NEW record => POST.
    + read => GET.
    + if the record exists then update else create a new record => PUT.
    + update/modify => PATCH.
    + delete => DELETE.

In [ ]:
from flask import Flask
from flask_restful import Api, Resource

app = Flask(__name__)
api = Api(app)


class MessageSender(Resource):
    def get(self):
        return (
            {'msg': 'Hello from flask'},  # response
            200,  # status code
            {'my_header': 'Playing with flask'}  # header 
        )

api.add_resource(MessageSender, '/') 
# to assign multiple url
# api.add_resource(HelloFresco, '/', '/home/', '/index/')

if __name__ == '__main__':
    app.run()

### Accessing REST API

* Now open a new terminal and send a GET request using curl as shown below.

```
$ curl http://127.0.0.1:5000/
```

* The above HTTP GET request is processed and response will be send.
* Or copy-paste the url to browser to see the response.
* It is also possible to access REST API using `request` module of `urllib` package.

```
from urllib import request
req = request.Request('http://127.0.0.1:5000/')

try:
    response = request.urlopen(req)
    print(response.read())
except urllib.error.HTTPError as e:
    print(e.code, e.read())
```


### Mapping Multiple URL's

* A single Resource can be mapped to one or more URL's.
* Multiple URL's can be passed to add_resource method of Api object.
```
api.add_resource(HelloFresco, '/', '/home/', '/index/')
```
* Now this url also work `http://127.0.0.1:5000/home/`

### Common Status Codes

* 1xx: Informational – Communicates transfer protocol-level information.
* 2xx: Success – Indicates that the client’s request was accepted successfully.
* 3xx: Redirection – Indicates that the client must take some additional action in order to complete their request.
* 4xx: Client Error – This category of error status codes points the finger at clients.
* 5xx: Server Error – The server takes responsibility for these error status codes.
<hr>

* 200 OK: Indicates that the request has succeeded.
* 201 Created: Indicates that the request has succeeded and a new resource has been created as a result.
<hr>

* 300 Multiple Choices: The request has more than one possible response. The user-agent or user should choose one of them.
* 301 Moved Permanently: The URL of the requested resource has been changed permanently. The new URL is given by the Location header field in the response. This response is cacheable unless indicated otherwise.
<hr>

* 400 Bad Request: The request could not be understood by the server due to incorrect syntax. The client SHOULD NOT repeat the request without modifications.
* 401 Unauthorized: Indicates that the request requires user authentication information. The client MAY repeat the request with a suitable Authorization header field
* 404 Not Found: The server can not find the requested resource.
* 405 Method Not Allowed: The request HTTP method is known by the server but has been disabled and cannot be used for that resource.
* 408 Request Timeout: Indicates that the server did not receive a complete request from the client within the server’s allotted timeout period.
<hr>

* 500 Internal Server Error: The server encountered an unexpected condition that prevented it from fulfilling the request.
* 501 Not Implemented: The HTTP method is not supported by the server and cannot be handled.
* 502 Bad Gateway: The server got an invalid response while working as a gateway to get the response needed to handle the request.
* 503 Service Unavailable: The server is not ready to handle the request.
* 504 Gateway Timeout: The server is acting as a gateway and cannot get a response in time for a request.

 ### Setting Response Status and Headers

* Below example shows how to set response, status and header:
```
    class HelloFresco(Resource):
      def get(self):
        return {'message': 'Welcome to Fresco Play!!!'}, 201, {'response_header1': 'some-message'}
```

* To see status code and header in browser: _right-click_ > _inspect_ > _network_
     ![browser_image_discovering_status_code](https://www.matthewedgar.net/wp-content/uploads/2023/06/image.png)
 
* The logic that Flask applies to converting return values into response objects is as follows:
    1. If a response object of the correct type is returned it’s directly returned from the view.
    2. If it’s a `string`, a response object is created with that data and the default parameters.
    3. If it’s a `dict`, a response object is created using jsonify.
    4. If a `tuple` is returned the items in the tuple can provide extra information. Such tuples have to be in the form `(response, status), (response, headers), or (response, status, headers)`. The status value will override the status code and headers can be a list or dictionary of additional header values.
    5. If none of that works, Flask will assume the return value is a valid WSGI application and convert that into a response object.

### Request Module

* Attributes available on the request object (_from flask import request_) during a request.
    * `request.data`: contains the incoming request data as string in case it came with a mimetype Flask does not handle.
    * `request.args`: the key/value pairs in the URL query string
    * `request.form`: the key/value pairs in the body, from a HTML post form, or JavaScript request that isn't JSON encoded
    * `request.files`: the files in the body, which Flask keeps separate from form. HTML forms must use enctype=multipart/form-data or files will not be uploaded.
    * `request.values`: combined args and form, preferring args if keys overlap
    * `request.json`: parsed JSON data. The request must have the application/json content type, or use request.get_json(force=True) to ignore the content type.

* All of these are MultiDict instances (except for json). You can access values using:
    * `request.form['name']`: use indexing if you know the key exists
    * `request.form.get('name')`: use get if the key might not exist
    * `request.form.getlist('name')`: use getlist if the key is sent multiple times and you want a list of values. get only returns the first value.

### Building a Basic CRUD Resource 

* Below code act as Server / Backend.
* It will receives the request and send the response.
* To hit the api or client-side approach:
    * Use Edge Browser extension `Boomerang` for sending get/put/post/delete request, same as postman.
    * It will send the request and receive the response.
    * Form should be used while sending request as in server-side `request.form` used.

In [ ]:
from flask import Flask, request
from flask_restful import Api, Resource, abort


# this will act as database
course_db = dict()

app = Flask(__name__)
api = Api(app)


class Courses(Resource):
    """
    Class that extends from Resource class, and having functionality to
    create, read, ulter and delete.
    """
    
    def post(self, course_id):
        """
        Post method to create new records.
        """
        if course_id in course_db.keys():
            abort(404, message="Course_Id {} already exists.".format(course_id))
        course_db[course_id] = request.form['course_name']
        return (
            {'message': f'Course created for course_id {course_id}: {course_db[course_id]}.'},
            201,
        )
    
    def get(self, course_id=None):
        """
        Get method to read the record based on course_id.
        """
        if course_id is None:
            return {'message': course_db}, 200
        elif course_id not in course_db.keys():
            abort(404, message=f"Course_Id: {course_id} not exists.")
        return (
            {'message': f'Course_id {course_id} is map to {course_db[course_id]}.'},
            200,
        )
    
    def put(self, course_id):
        """
        Put method to alter the exist record based on course_id.
        """
        if course_id not in course_db.keys():
            abort(404, message=f"Course_Id: {course_id} not exists.")
        course_db[course_id] = request.form['course_name']
        return (
            {'message': f'Course alter for course_id {course_id}: {course_db[course_id]}.'},
            200,
        )
    
    def delete(self, course_id):
        """
        Delete method to remove the record based on course_id.
        """
        if course_id not in course_db.keys():
            abort(404, message=f"Course_Id: {course_id} not exists.")
        course_name = course_db.pop(course_id)
        return (
            {'message': f'Course {course_name} with course_id {course_id}, removed successfully.'},
            200,
        )
        
        
api.add_resource(Courses, '/Courses/', '/Courses/<int:course_id>')


if __name__ == '__main__':
    app.run()

### Parsing Requests

* We will see how to validate the data sent over a HTTP request using `reqparse` module of `flask_restful`.
* The utilities of reqparse module are modelled based on `argparse` module.
* For using reqparse, an instance of `RequestParser` class has to be created first.
* Later, expected data arguments have to be added to created parser.
* Many parameters, such as type, required, help, location can be set while adding an argument.
* Value associated with any argument can be accessed using dictionary, returned by `parse_args` method of created instance.
* An example of creating parser and adding an argument argument1 is shown below.
```
    from flask_restful import reqparse

    parser = reqparse.RequestParser()
    parser.add_argument('argument1', type=int, help='This argument must be an integer', location='form')
    args = parser.parse_args() # 'args' is a dictionary
```
* Suppose below code is server-side code and if client send `principal_amount='hello'`. Since hello cannot be converted to a float type, the response would thrown an error as `500 b'{"message": "Internal Server Error"}\n'`
```
    from flask import Flask, request
    from flask_restful import Api, Resource, abort

    app = Flask(__name__)
    api = Api(app)

    class SimpleInterest(Resource):
        def post(self):
            principal = float(request.form['principal_amount'])  # if principal_amount='hello', error will be thrown
            rate = float(request.form['rate'])
            time = int(request.form['time'])
            return {'simple_interest': principal*rate*time/100.0}

    api.add_resource(SimpleInterest, '/SimpleInterest')
```
* By default, RequestParser aborts when it encounters the first error.
* However, it is possible to combine all errors together and send them at once to client.
* It can be achieved by setting `bundle_errors` parameter of RequestParser class to True, while creating it's object, as shown below.
```
parser = reqparse.RequestParser(bundle_errors=True)
```
* In order to validate the data, passed via HTTP requests, let's modify definition of SimpleInterest resource as shown below.
* Now if client send principal_amount='hello' and time=5.5, meaningfull error will be thrown 
```
400 b'{"message": {"time": "No. of Years must be an integer", "principal_amount": "Principal amount must be a number"}}\n'
```
* If an argument needs to take any one value from a list of defined choices, it can be done using choices parameter of add_argument method.
```
parser.add_argument(
    'year',
    choices=('2017', '2018', '2019', '2020'),
    help='Bad choice', location='form'
)
```

In [ ]:
from flask import Flask, request
from flask_restful import Api, Resource, reqparse, abort

app = Flask(__name__)
api = Api(app)

parser = reqparse.RequestParser(bundle_errors=True)
parser.add_argument('principal_amount', type=float, help='Principal amount must be a number', location='form')
parser.add_argument('time', type=int, help="No. of Years must be an integer", location='form')
parser.add_argument('rate', type=float, help='Rate must be a number', location='form', required=True)


class SimpleInterest(Resource):
    def post(self):
        args = parser.parse_args()
        principal = args['principal_amount']
        time = args['time']
        rate = args['rate']
        return {'simple_interest': principal*rate*time/100.0}

api.add_resource(SimpleInterest, '/SimpleInterest')

if __name__ == '__main__':
    app.run()

#### Parser Inheritance

* Many a times, you may define a different parser for each defined resource.
* Some of the resources may have common arguments.
* In such a scenario, it is good to define a parent parser having shared argument details.
* The parent parser can be further extended using `copy` method.
* Any argument in parent parser can be overwritten using `replace_argument` method and can be completely removed using `remove_argument` method.
```
    parser = reqparse.RequestParser()
    parser.add_argument('arg1', type=int)

    parser_copy = parser.copy()
    parser_copy.add_argument('arg2', type=int)

    parser_copy.replace_argument('arg1', required=True)
```

### Formatting Response Data

* By default, if the iterable returned by any of the methods defined in a Resource class is a Python Data Structure, then it is rendered as it is.
* However, if an other object is returned, the rendering may not be as expected.
* As a reason, `flask_restful` provides `fields` module to specify the structure of response.
* Structure of a response must be either a dictionary or an Ordered dictionary.
* Different Field Types:
    | Field Name | Description |
    | :- | :- |
    | fields.Raw | Base field class, from which custom fields can be derived
    | fields.String | Outputs a value as String
    | fields.Float | Outputs a value as Float
    | fields.Integer | Outputs a value as Integer
    | fields.Boolean | Outputs a value as a Boolean
    | fields.Url | Outputs an URL in a string form
    | fields.DateTime | Returns a formated datetime string in UTC


#### Marshalling

* Pickling and marshalling is almost same in python. It is used in serializing and deserializing a Python object structure or in other words the process of converting a Python object into a byte stream.
* Flask-restful also comes with methods like `marshal`, `marshal_with` and `marshal_with_field`, which are useful for filtering the response data from return values of methods.
* `marshal` method of Flask-RESTful package provides the ability to filter data of specified fields in a desired format.
* `marshal` method takes either a dictionary, list, or an object as input of data and a dictionary of fields, to be available in output, and filters the data based on those fields.
* In addition to marshal method, you can also use `marshal_with` decorator for marshalling the return object of a method with specified data fields.
* `marshal_with_field` is an another decorator which formats return values of methods with a single field.


#### Renaming Attributes

* Any field name, displayed to users, can be altered from it's internal field name by renaming it.
* Renaming an internal field name can be achieved by setting `attribute` parameter value, of any `field` type, to the _new public_ field name.
* An example of using attribute is shown below. It renames field name, private_name to public_name.
```
fields = {
    'public_name': fields.String(attribute='private_name'),
    'address': fields.String,
    'age':fields.Integer,
}
```


#### Custom Fields

* fields module can also be used to create custom fields.
* A custom field has to be derived from `fields.Raw` class and must contain definition of format method.
* In below example, LowerCase class is derived from fields.Raw class and it's format method transforms an input value into lower case.
```
class LowerCase(fields.Raw):
    def format(self, value):
        return value.lower()
```

In [ ]:
from flask_restful import marshal, marshal_with, marshal_with_field, fields


data = {
    'age': 36,
    'name': 'Philips'
}

required_fields = {
    'name': fields.Raw
}

print('marshal:', marshal(data, required_fields))



@marshal_with(required_fields)
def get():
    return data

print('marshal_with:', get())


@marshal_with_field(fields.List(fields.Integer))
def get():
    return ['2', 5.023, 199]

print('marshal_with_field:', get())

In [ ]:
from flask import Flask
from datetime import datetime
from flask_restful import Api, Resource, reqparse, request, marshal_with, fields


app = Flask(__name__)
api = Api(app)

parser = reqparse.RequestParser()
parser.add_argument('SI', type=float, required=True, location='form', help='Provide Simple Interest amount.')

req_fields = {
    'si_public': fields.Float(attribute='si_private'),
    'computed_date': fields.DateTime(dt_format='rfc822')
}


class SimpleInterest(Resource):
    @marshal_with(req_fields)
    def post(self):
        args = parser.parse_args()
        si = args['SI']
        return {
            'si_private': si,  # in response si_public would be visible
            'computed_date': datetime.now()
        }

api.add_resource(SimpleInterest, '/simple')


if __name__ == '__main__':
    app.run()

### Defining Constructor Parameters

* In case if any resource has external dependencies, those dependencies can be passed to constructor method of the resource.
* The below example shows definition of a sample Resource Task. The `__init__` method initializes `self.db` to a database connection object and get method uses execute method associated with connection object.
```
    from flask_restful import Resource

    class Task(Resource):
        def __init__(self, **kwargs):
            self.db = kwargs['dbconnection']

        def get(self):
            return self.db.execute()
```
* The way of passing database connection object to Task is shown below.
```
    db_connection = db.Connection()

    api.add_resource(TodoNext, '/next',
        resource_class_kwargs={ 'dbconnection': db_connection })
```

### Structuring a Project

* A Flask-restful app can be organised in multiple ways.
* In this topic we will see one way of organizing an app, which is scalable well with larger apps.
* In general, a project app is split into three sections: `routes`, `resources`, and `common` sections.
* Files under `routes` section contains the app and definition of various routes.
* Files under `resources` section contain definition of various resources.
* Finally, files under `common` section contain definition of functions used across the application.
```
    myprojectapi/
         __init__.py
         app.py
         resources/
            __init__.py 
            hellofresco.py 
            frescoplaycourses.py
            simpleinterest.py
         common/
            __init__.py
            utils.py
```
* The routes section contains the file app.py. Add the below shown content to it.
* It defines a Flask application, app and a restful Api, api. The api is linked to application app.
* Further the resources are imported from files in resources folder and added to api .
```
    from flask import Flask
    from flask_restful import Api
    from myprojectapi.resources.hellofresco import HelloFresco
    from myprojectapi.resources.frescoplaycourses import PlayCourses
    from myprojectapi.resources.simple_interest import SimpleInterest

    app = Flask(__name__)

    api = Api(app)
    api.add_resource(HelloFresco, '/')
    api.add_resource(PlayCourses, '/Courses/', '/Courses/<int:course_id>')
    api.add_resource(SimpleInterest, '/simpleinterest/')
```
* Files containing the definition of resources are placed in myprojectapi/resources folder .
* Remove the following lines from all the three resource files.
    * Lines defining a flask app i.e `app = Flask(__name__)`.
    * Lines defining a flask restful api i.e `api = Api(app)`.
    * Lines adding resources to a api using `add_resource` method.
    * Block of lines used for running the applictaion i,e `app.run()`.
* Required resources can be imported in myprojectapi/app.py and used.
* Any new resource can be directly added to resources folder.